In [34]:
%%sh
pip install openai

Defaulting to user installation because normal site-packages is not writeable


In [35]:
from openai import OpenAI
from IPython.core.display import display, HTML
client = OpenAI(api_key="")

/tmp/ipykernel_17772/3613881004.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [102]:
system_prompt = "Begin every response with `<!DOCTYPE html>` and immediately proceed to construct a functional HTML document that includes a complete form. The form must be ready for immediate use, catering to the user's request with a comprehensive range of form elements like sliders, checkboxes, radio buttons, dropdowns, text areas, and specifically, an 'Others' field for any unspecified options. DO NOT generate HTML code with placeholders, comments suggesting the addition of more options, or any indication that the form is unfinished or requires further modification by the user. Your output must represent a complete, error-free HTML document that creatively fulfills the requested functionalities or content, ensuring a practical and engaging form interface that anticipates various user inputs without directly using phrases from the prompt."
response = client.chat.completions.create(
  model="gpt-4-0125-preview",
  # model="gpt-3.5-turbo-0125",
  messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "mood tracker journal entry"},
    # {"role": "user", "content": "doctor booking site, with fields asking about additional information a algorithm might need from users to better suggest doctors"},
  ]
)

In [100]:
print(response.choices[0].message.content)

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Mood Tracker Journal Entry</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 20px;
            padding: 20px;
            background-color: #f0f0f0;
        }
        .form-wrapper {
            background-color: #ffffff;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
        }
        .form-group {
            margin-bottom: 20px;
        }
        label {
            display: block;
            margin-bottom: 10px;
        }
        input[type="text"],
        textarea {
            width: 100%;
            padding: 10px;
            margin-top: 5px;
            border: 1px solid #dddddd;
            border-radius: 5px;
            box-sizing: border-box;
        }
        .form-control {
            margin-top: 5px;
   

In [103]:
display(HTML(response.choices[0].message.content))

In [112]:

user_request = "why am I feeling so unhappy today?"

system_prompt = """
Prompt: "Analyze the user's request and determine if additional information is needed to fulfill it. If additional information is required, identify what is missing and indicate that more information is needed by setting 'needs_additional_information' to True. Then, provide a short, detailed description for a form interface that allows the user to input all of the additional information. Ensure the response strictly adheres to the instructions, without conversational transitions, explanations, working notes, steps, polite introductions, or conclusions."

User's Request: "{}"

---

[If additional information is required]

missing_details: [list_missing_details_here]
needs_additional_information = True

[Form Interface Description]
Generate a modern form element for user's request. The interface should be straightforward and contain rows of fields as well as a submit button. It will ask for the following fields in the given order: [list_of_fields_here].

[If no additional information is required]

missing_details: None
needs_additional_information = False

---

Example 1: User's Request: "I want to book a flight."

Output:

missing_details: date, origin, destination, number of stops, price range
needs_additional_information = True

[Form Interface Description]
Generate a modern form element for booking flights. The interface should be straightforward and contain rows of fields as well as a submit button. It will ask for the following fields in the given order: origin, destination, is round trip (checkbox), date, number of stops (dropdown: any, non-stop, 1 stop, 2+ stops), price range slider.


""".format(user_request)

# system_prompt = "Begin every response with `<!DOCTYPE html>` and immediately proceed to construct a functional HTML document that includes a complete form. The form must be ready for immediate use, catering to the user's request with a comprehensive range of form elements like sliders, checkboxes, radio buttons, dropdowns, text areas, and specifically, an 'Others' field for any unspecified options. DO NOT generate HTML code with placeholders, comments suggesting the addition of more options, or any indication that the form is unfinished or requires further modification by the user. Your output must represent a complete, error-free HTML document that creatively fulfills the requested functionalities or content, ensuring a practical and engaging form interface that anticipates various user inputs without directly using phrases from the prompt."
response = client.chat.completions.create(
  model="gpt-4-0125-preview",
  # model="gpt-3.5-turbo-0125",
  messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_request},
    # {"role": "user", "content": "doctor booking site, with fields asking about additional information a algorithm might need from users to better suggest doctors"},
  ]
)

In [162]:

user_request = "book uber!"

system_prompt = """
[SYSTEM PROMPT]

1. Analyze the user's query:
   - If the query is straightforward (e.g., a direct question about facts or knowledge), respond directly.
   - If the query requires an action (e.g., summarizing text, booking a flight, logging a journal entry), proceed to step 2.

2. Determine action requirements:
   - List the information provided 'Provided Details', write None if no additional information provided.
   - List additional information needed but missing 'Missing Details'.
   - If additional information is needed, set "needs_additional_information = True".

3. Request additional information (if needed):
   - If "needs_additional_information = True", generate a description for a form interface tailored to the action, detailing the required fields for the user to fill in.

[USER QUERY HANDLING]

- For direct inquiries (e.g., "Why do dogs wag their tails?"), provide a normal answer under 'Response' without further action, as a language model would normally.
- For action-based queries (e.g., "Book me a flight"), analyze the completeness of the information provided:
  - If list missing details (e.g., date, origin, destination), output "needs_additional_information = True", and describe a form interface for information collection.
  - If complete, proceed with the action as per the model's capabilities.

[FORM INTERFACE DESCRIPTION]

- Upon determining that additional information is needed, generate a concise description of a form interface specific to the request. For example, for flight booking, detail a form requesting origin, destination, trip type, date, number of stops, and a price range slider, in a structured and modern design, emphasizing straightforwardness and usability.

[OUTPUT RESTRICTIONS]

- Do not include conversational transitions, explanations, working notes, steps to execute, or polite introductions/conclusions in responses.
- Focus solely on analyzing the query, determining if additional information is required, and, if so, requesting that information through a structured form description.

[IMPLEMENTATION NOTES]

- Implement logic to distinguish between types of requests (informational vs. action-oriented).
- For action-oriented requests, develop a checklist of information typically required for different actions (e.g., flight booking, summary generation) and use this to assess completeness.
- Design form interface descriptions to be clear, concise, and user-friendly, ensuring they directly address the information gaps identified.

---

### Good Examples

User Query: "Book me a flight from New York to London next Friday."

System Response:

Analysis: The query requires action (flight booking).
Provided Details: Origin, destination, date
Missing Details: Number of stops, price range.
needs_additional_information = True
Form Interface Description: "To complete your flight booking, please provide the following details: Number of stops (choose from 'Non-stop', '1 stop', '2+ stops'), and Price Range (use the slider to set your minimum and maximum budget). Once you've filled these out, click the 'Submit' button to proceed with your booking."
This response is good because it directly addresses the user's request, identifies the action required, determines that additional information is needed, and provides a clear, concise description of the form interface for the user to complete the missing details.

User Query: "Why do dogs wag their tails?"

System Response:

Analysis: The query requires a response but no action required.
Response: Dogs wag their tails primarily as a form of communication. This behavior can indicate various emotions, such as happiness, excitement, and even agitation. Tail wagging is a dog's way of expressing their feelings and intentions to both their human companions and other animals.
This response is good because it directly answers the user's straightforward question without requiring any further action or additional information. It provides a concise and informative explanation, adhering to the guidelines of focusing solely on delivering the information requested.

User Query: "Good morning!"

System Response:

Analysis: The query requires a response but no action required.
Response: Hello! How can I assist you today?
This response is good because it responds to the user as a language model would, since no action or request is required.


### Bad Examples

User Query: "Book me a flight from New York to London next Friday."

System Response:

Hi there! So you're looking to fly from New York to London next Friday? That sounds like a fantastic trip! Booking flights is something I can help with, but I'll need a bit more info to get started. You see, when booking flights, it's crucial to have all the details like how many stops you'd prefer, what your budget looks like, and whether you're flexible with your dates. Could you provide those details? Also, don't forget to tell me if you're looking for round-trip tickets or just a one-way journey. Once I have all this, we can start looking into your options and get everything sorted for your trip!
This response is bad because it violates the output restrictions: it includes conversational transitions, explanations, and does not directly lead to a structured form interface description for the missing details. It also adds unnecessary information and questions not streamlined by the prompt design, making it less efficient and more confusing for the user.


User Query: "Book me a flight"

System Response:

Analysis: The query requires action (flight booking).
Provided Details: None
needs_additional_information = True
Form Interface Description: "To complete your flight booking, please provide the following details: Number of stops (choose from 'Non-stop', '1 stop', '2+ stops'), and Price Range (use the slider to set your minimum and maximum budget). Once you've filled these out, click the 'Submit' button to proceed with your booking."
This response is bad because the output is missing several of the fields required by the instructions: Missing Details. These intermitten fields are mandatory for good output and train of thought reasoning. The output Form Interface Description is also missing several information required for a flight booking like origin, destination and date.


User Query: "hey"

System Response:

Analysis: The user query is unclear and does not provide a specific request or action.
Response: Please provide more information or specify what you need assistance with to better address your query.
This response is bad because if the user message was not a request or action, the model should have answered as a language model should have.
"""

# system_prompt = "Begin every response with `<!DOCTYPE html>` and immediately proceed to construct a functional HTML document that includes a complete form. The form must be ready for immediate use, catering to the user's request with a comprehensive range of form elements like sliders, checkboxes, radio buttons, dropdowns, text areas, and specifically, an 'Others' field for any unspecified options. DO NOT generate HTML code with placeholders, comments suggesting the addition of more options, or any indication that the form is unfinished or requires further modification by the user. Your output must represent a complete, error-free HTML document that creatively fulfills the requested functionalities or content, ensuring a practical and engaging form interface that anticipates various user inputs without directly using phrases from the prompt."
response = client.chat.completions.create(
  # model="gpt-4-0125-preview",
  model="gpt-3.5-turbo-0125",
  messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_request},
    # {"role": "user", "content": "doctor booking site, with fields asking about additional information a algorithm might need from users to better suggest doctors"},
  ]
)

In [163]:
print(response.choices[0].message.content)

Analysis: The query requires action (booking an Uber).
Provided Details: None
Missing Details: Pickup location, destination, ride type (e.g., UberX, UberXL), date/time.
needs_additional_information = True

Form Interface Description: "To book your Uber ride, please provide the following details: Pickup Location (address or coordinates), Destination (address or coordinates), Ride Type (choose from UberX, UberXL, etc.), Date/Time for the ride. Once you've filled out these details, click 'Book' to confirm your ride."


In [164]:
system_prompt = "Begin every response with `<!DOCTYPE html>` and immediately proceed to construct a functional HTML document that includes a complete form. The form must be ready for immediate use, catering to the user's request with a comprehensive range of form elements like sliders, checkboxes, radio buttons, dropdowns, text areas, and specifically, an 'Others' field for any unspecified options. DO NOT generate HTML code with placeholders, comments suggesting the addition of more options, or any indication that the form is unfinished or requires further modification by the user. Your output must represent a complete, error-free HTML document that creatively fulfills the requested functionalities or content, ensuring a practical and engaging form interface that anticipates various user inputs without directly using phrases from the prompt."
response = client.chat.completions.create(
  model="gpt-4-0125-preview",
  # model="gpt-3.5-turbo-0125",
  messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": response.choices[0].message.content},
  ]
)

In [165]:
print(response.choices[0].message.content)

<!DOCTYPE html>
<html>
<head>
    <title>Book Your Uber Ride</title>
    <style>
        body {
            font-family: Arial, sans-serif;
        }
        .form-group {
            margin-bottom: 15px;
        }
        label {
            display: block;
            margin-bottom: 5px;
        }
        .form-control {
            width: 100%;
            padding: 10px;
            margin-bottom: 10px;
            border: 1px solid #ccc;
            border-radius: 4px;
        }
        .form-submit {
            background-color: #007bff;
            color: white;
            padding: 10px 20px;
            border: none;
            border-radius: 4px;
            cursor: pointer;
        }
        .form-submit:hover {
            background-color: #0056b3;
        }
    </style>
</head>
<body>
    <h2>To book your Uber ride, please provide the following details:</h2>
    <form>
        <div class="form-group">
            <label for="pickupLocation">Pickup Location (address or co

In [166]:
display(HTML(response.choices[0].message.content))

In [138]:
user_prompt = """
how do i export a string from one python file and import it somewhere else?
I don't want to make a def function, can I just export the string itself like i can in javascript?
"""
_response = client.chat.completions.create(
  model="gpt-4-0125-preview",
  # model="gpt-3.5-turbo-0125",
  messages=[
    {"role": "system", "content": ""},
    {"role": "user", "content": user_prompt},
    # {"role": "user", "content": "doctor booking site, with fields asking about additional information a algorithm might need from users to better suggest doctors"},
  ]
)
print(_response.choices[0].message.content)

In Python, you can share variables, including strings, across different files/modules without wrapping them into functions, akin to how you might export and import variables in JavaScript. While Python doesn't use the `export` and `import` syntax in the same way JavaScript does, you can achieve similar functionality using Python's import system.

### Exporting a String from a Module

Suppose you have two Python files: `module_a.py` and `module_b.py`. You want to define a string in `module_a.py` and use it in `module_b.py`.

**`module_a.py`:**

```python
# This is the string you want to export
my_string = "Hello, this is a string from module_a!"
```

### Importing the String in Another Module

**`module_b.py`:**

To use `my_string` in `module_b.py`, you import it directly from `module_a.py`.

```python
# Importing the specific variable my_string from module_a
from module_a import my_string

# Now you can use my_string in this file
print(my_string)
```

**Note:** When you run `module_b.p

In [167]:

def format_html(html: str) -> str:
    from bs4 import BeautifulSoup

    # Using BeautifulSoup to parse and prettify the HTML, which will also remove unnecessary whitespace
    soup = BeautifulSoup(html, 'html.parser')
    formatted_html = soup.prettify()

    # Splitting the formatted HTML into lines
    lines = formatted_html.split('\n')

    # Removing empty lines
    non_empty_lines = [line for line in lines if line.strip() != '']

    # Joining the non-empty lines back into a string
    cleaned_html = '\n'.join(non_empty_lines)

    return cleaned_html
